In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    read_full_cli_df,
    get_visualization_subtitle,
    get_country_color_map,
    get_siteid_color_maps,
    apply_theme,
)
from web import for_website

alt.data_transformers.disable_max_rows() # Allow using rows more than 5000

In [ ]:
df = read_full_cli_df()
df

In [ ]:
pdf = df

# Negative values to zeros
pdf.loc[pdf['num_patients_all_still_in_hospital'] < 0, 'num_patients_all_still_in_hospital'] = np.nan
pdf.loc[pdf['num_patients_ever_severe_still_in_hospital'] < 0, 'num_patients_ever_severe_still_in_hospital'] = np.nan

# add never severe values
pdf['num_patients_never_severe_in_hospital'] = pdf['num_patients_all_still_in_hospital'] - pdf['num_patients_ever_severe_still_in_hospital']

pdf = pdf.rename(columns={
    'num_patients_all_still_in_hospital': 'all',
    'num_patients_ever_severe_still_in_hospital': 'ever severe',
    'num_patients_never_severe_in_hospital': 'never severe'
})

# Wide to long
pdf = pd.melt(pdf, id_vars=[
    'siteid', 
    'days_since_admission',
    'color',
    'country'
])
pdf = pdf.rename(columns={"variable": 'category', "value": 'num_patients'})

pdf

In [ ]:
COUNTRY_NAMES = list(get_country_color_map().keys())
COUNTRY_COLORS = list(get_country_color_map().values())
COUNTRY_COLOR_MAP = {COUNTRY_NAMES[i]: COUNTRY_COLORS[i] for i in range(len(COUNTRY_NAMES))}

COUNTRY_COLOR_MAP

In [ ]:
SITE_NAMES = list(get_siteid_color_maps().keys())
SITE_COLORS = list(get_siteid_color_maps().values())
SITE_COLOR_MAP = {SITE_NAMES[i]: SITE_COLORS[i] for i in range(len(SITE_NAMES))}

SITE_COLOR_MAP

In [ ]:
def line(
    df=None,
    c_field=None,
    c_domain=None,
    c_range=None,
    x_field=None,
    y_field=None,
    title=''
):        
    # Selections
    legend_selection = alt.selection_multi(fields=[c_field], bind="legend")
    nearest = alt.selection(type="single", nearest=True, on="mouseover", encodings=["x", "y"], empty='none', clear="mouseout")

    # Rule
    rule = (
        alt.Chart(df)
            .mark_rule(color="red")
            .encode(x=f"{x_field}:Q", size=alt.value(0.5))
            .transform_filter(nearest)
    )
    
    line = (
        alt.Chart(df)
            .mark_line(size=3)
            .encode(
                x=alt.X(
                    f"{x_field}:Q", 
                    scale=alt.Scale(nice=False, clamp=False, padding=10), 
                    title="Days since positive"
                ),
                y=alt.Y(
                    f'sum({y_field}):Q', 
                    axis=alt.Axis(format="r"),
                    scale=alt.Scale(nice=False, clamp=False, padding=10),
                    title="Number of patients"
                ),
                color=alt.Color(c_field, scale=alt.Scale(domain=c_domain, range=c_range), title=None),
                opacity=alt.value(0.7),
                tooltip=[
                    alt.Tooltip(c_field, title="Country"),
                    alt.Tooltip(f'sum({y_field}):Q', title="Number of patients", format="r"),
                    alt.Tooltip(x_field, title="Days since positive")
                ]
            )
    )
    
    circle_size = 10
    circle = (
        line
            .mark_circle(size=circle_size, opacity=0.7)
            .encode(size=alt.condition(~nearest, alt.value(circle_size), alt.value(circle_size * 2)))
            .add_selection(nearest)
    )

    plot = (
        (line + circle.add_selection(nearest) + rule)
        .transform_filter(alt.FieldOneOfPredicate(field=c_field, oneOf=c_domain))
        .transform_filter(legend_selection)
        .add_selection(legend_selection)
        .properties(
            title={
                "text": title,
                "subtitle": get_visualization_subtitle(data_release='06-16-2020'),
                "subtitleColor": "gray",
            },
            width=500, height=400
        )
        .interactive()
    )

    return plot

In [ ]:
ever = line(
    df=pdf[pdf['category'] == 'ever severe'], 
    c_field='country',
    c_domain=list(df['country'].unique()),
    c_range=list(df['color'].unique()),
    x_field='days_since_admission',
    y_field='num_patients',
    title='Ever Severe Patients Still In Hospital'
)

never = line(
    df=pdf[pdf['category'] == 'never severe'], 
    c_field='country',
    c_domain=list(df['country'].unique()),
    c_range=list(df['color'].unique()),
    x_field='days_since_admission',
    y_field='num_patients',
    title='Never Severe Patients Still In Hospital'
)

final = apply_theme((ever | never).resolve_scale(y='shared'), legend_orient="right")

for_website(final, "clinicalcourse1.1", "clinicalcourse1.1")

In [ ]:
fpdf = pdf[pdf['category'] != 'all']

plot = line(
    df=fpdf,
    c_field='category',
    c_domain=['ever severe', 'never severe'],
    c_range=['#DC3A11', '#3366CC'],
    x_field='days_since_admission',
    y_field='num_patients',
    title='Ever and Never Severe Patients Still In Hospital'
)

apply_theme(plot.resolve_scale(y='shared'), legend_orient="right")